<a href="https://colab.research.google.com/github/pravinkedia/PYTHON_ML/blob/main/Boosting_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[statinfer.com](https://statinfer.com/)

# Packages

In [ ]:
import pandas as pd
import sklearn as sk
import numpy as np
import scipy as sp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import time
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing

# Data Importing

Dataset Background : https://www.kaggle.com/c/otto-group-product-classification-challenge/notebooks

In [ ]:
ecom_prod_train=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/train.csv")
ecom_prod_test=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/test.csv")

In [ ]:
print(ecom_prod_train.sample(10))
print(ecom_prod_test.sample(10))

In [ ]:
#Frequency counts of the products
ecom_prod_train['Category'].value_counts()

# Prepare Features, Train and Test data

In [ ]:
features=list(ecom_prod_train.columns[1:101])
print("features \n", features)
X_train=ecom_prod_train[features]
y_train=ecom_prod_train['Category']

X_test=ecom_prod_test[features]
y_test=ecom_prod_test['Category']

# Decision Tree model

In [ ]:
###buildng Decision tree on the training data ####
from sklearn import tree
tree = tree.DecisionTreeClassifier(max_depth=9)
tree.fit(X_train,y_train)

## Decision Tree Results

In [ ]:
#Accuracy on train data
print("Decision Tree Results \n")
print("Accuracy on train data" , tree.score(X_train, y_train))
print("Accuracy on test data" , tree.score(X_test, y_test))

# Boosted Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost=GradientBoostingClassifier(n_estimators=100,verbose=1) #Number of iterations try n_estimators=[20,100,200]
##fitting the gradient boost classifier
start_time = time.time()
boost.fit(X_train,y_train)
print("Time taken by GBM "+ str((time.time() - start_time))+ " Seconds")

## GBM Results

In [ ]:
###predicting Gradient boosting model on the train Data
boost_predict_train=boost.predict(X_train)
cm1 = confusion_matrix(y_train,boost_predict_train)
print(cm1)

accuracy_train=f1_score(y_train, boost_predict_train, average='micro') 
print("train accuracy", accuracy_train)

In [ ]:
###predicting Gradient boosting model on the test Data
boost_predict_test=boost.predict(X_test)
cm1 = confusion_matrix(y_test,boost_predict_test)
print(cm1)

accuracy_test=f1_score(y_test, boost_predict_test, average='micro') 
print("test accuracy", accuracy_test)

# XGB Model

In [ ]:
#Creating XGB Friendly data and matrices
train_labels = y_train.values
train_labels = preprocessing.LabelEncoder().fit_transform(train_labels)
test_labels = y_test.values
test_labels = preprocessing.LabelEncoder().fit_transform(test_labels)

matrix_train = xgboost.DMatrix(X_train,label=train_labels)
matrix_test = xgboost.DMatrix(X_test,label=test_labels)

In [ ]:
params = {
    'max_depth': 8, 
    'eta':0.02, #Learning Rate
    'eval_metric':'merror', # Multiclass classification error rate. 
    'tree_method' : "gpu_hist", # use this with colab in gpu mode for faster training
    'num_class': 9
}

start_time = time.time()

model=xgboost.train(params=params,
                    dtrain=matrix_train,
                    num_boost_round=300,    #Number of trees
                    early_stopping_rounds=3, # Stop after 5 rounds, if test error doesn't improve. 
                    evals=[(matrix_test,'test')] 
                   )

print("Time taken by XGB "+ str((time.time() - start_time))+ " Seconds")

## XGB Results

In [ ]:
###prediction using XGB on the train Data
boost_predict_train=model.predict(matrix_train)
cm1 = confusion_matrix(train_labels,boost_predict_train)
print(cm1)

accuracy_train=f1_score(train_labels, boost_predict_train, average='micro') 
print("train accuracy", accuracy_train)

In [ ]:
###prediction using XGB on the test Data
boost_predict_test=model.predict(matrix_test)
cm1 = confusion_matrix(test_labels,boost_predict_test)
print(cm1)

accuracy_test=f1_score(test_labels, boost_predict_test, average='micro') 
print("test accuracy", accuracy_test)